# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core import ScriptRunConfig

from azureml.train.hyperdrive.parameter_expressions import normal, choice

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from azureml.core.model import Model

import os
import joblib


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Detailed notes provided in the project file automl.ipynb

In [2]:
ds = TabularDatasetFactory.from_delimited_files('https://github.com/dntrply/nd00333-capstone/raw/master/dataset/Breast_cancer_data.csv')
df = ds.to_pandas_dataframe()
df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [3]:
df.describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,1.000000


In [26]:
ws = Workspace.from_config()

experiment=Experiment(ws, 'experiment-capstone-hyper')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# Use https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py as a reference
#

# See if it already exists
# this ofcourse helps if you are reunning the notebook from teh start and do not
# need to recreate the compute cluster
try:
    compute_cluster = ComputeTarget(workspace=ws, name='COMPUTE-HYPER')
    # No exception thrown - Found it - use it below
except ComputeTargetException:
    # Did not find the compute target - will need to create one
    # Specify the compute cluster configuration first
    # See https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target
    # and after clicking on Dv2
    # see https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target

    # CPU cluster instance - from the Microsoft recommended options
    cc_cfg = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS3_v2',
                                                            max_nodes=4,
                                                            description='CPU Compute Cluster created programatically for HyperParameter model')

    # Next, create the cluster
    compute_cluster = ComputeTarget.create(workspace=ws, name='COMPUTE-HYPER', provisioning_configuration=cc_cfg)

# We have a compute cluster - either newly created - or created earlier
# We may wait for the create operation to complete
compute_cluster.wait_for_completion(show_output=True)

Workspace name: quick-starts-ws-162691
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-162691
InProgress.AmlCompute is getting created. Consider calling wait_for_completion() first

...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The train.py script uses [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression) from scikit-learn. The task is to maximize accuracy. 

### The parameterts to the model are:
Inverse of regularization strength : --C<br/>
Max Iterations : --max_iter

A random combination of values for these parameters will be iterated over to determine the optimal values for these two hyper parameters.
<br/><br/>

[Bandit Policy](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py) defines an early termination policy based on slack criteria (slack-factor)
<br/><br/>
The primary metric is accuracy, inferred by scikit-learn LogisticRegression


In [5]:
# BanditPolicy is a type of EarlyTerminationPolicy
# More info can be found at 
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

# from train.py - we know that the parameters are --C and --max_iter
# train.py uses the scikit-learn LogisticRegression model

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10),
    "--max_iter": choice(50, 100, 200, 300, 400),
})

# We need to specify/provide an environment
# We could create our own environment with corresponding yaml file
# We may also provide an environment 'out-of-the-box' from one of the many environments provided

# to install required packages
environment = Environment('myenv')
dependencies = CondaDependencies.create(
    pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], 
    conda_packages = ['scikit-learn==0.22.1'])

environment.python.conda_dependencies = dependencies

# SKLearn is deprecated - instead we specity the run configuration using ScriptRunConfig
# It is necessary to pass the directory, the python script, the compute cluster and the environment

#TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>


run_config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_cluster,
    environment=environment,
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    run_config=run_config,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
)

In [6]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [8]:
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_83c46fb8-a462-4bff-818c-a1454623c5a4
Web View: https://ml.azure.com/runs/HD_83c46fb8-a462-4bff-818c-a1454623c5a4?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-162691/workspaces/quick-starts-ws-162691&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-11-03T23:19:30.696186][API][INFO]Experiment created<END>\n""<START>[2021-11-03T23:19:31.158211][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2021-11-03T23:19:31.400704][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_83c46fb8-a462-4bff-818c-a1454623c5a4
Web View: https://ml.azure.com/runs/HD_83c46fb8-a462-4bff-818c-a1454623c5a4?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-162691/workspaces/quick-starts-ws-162691&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_83c46fb8-a462-4bff-818c-a1454623c5a4',
 'target': 'COMPUTE-HYPER',
 'status': 'Completed',
 'startTimeUtc': '2021-11-03T23:19:30.474877Z',
 'endTimeUtc': '2021-11-03T23:37:22.735878Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0782d7a1-0f00-41b3-8fbb-de3a7be23b10',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.9385964912280702',
  'best_child_run_id': 'HD_83c46fb8-a462-4bff-818c-a1454623c5a4_7',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg162691.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_83c46fb8-a462-4bff-818c-a1454623c5a4/azureml-logs/hyperdrive.txt?s

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
# Get your best run and save the model from that run.


### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the model hyperparameter values corresponding to the best run
print(f'********** {best_hyperdrive_run.get_details()["runDefinition"]["arguments"]}')
print(best_hyperdrive_run.get_details())

# Get the accuracy corresponsding to the best run
print(f'********** Best HyperDrive run Accuracy: {best_hyperdrive_run.get_metrics()["Accuracy"]}')

********** ['--C', '10', '--max_iter', '400']
{'runId': 'HD_83c46fb8-a462-4bff-818c-a1454623c5a4_7', 'target': 'COMPUTE-HYPER', 'status': 'Completed', 'startTimeUtc': '2021-11-03T23:32:10.288493Z', 'endTimeUtc': '2021-11-03T23:34:30.926476Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '0782d7a1-0f00-41b3-8fbb-de3a7be23b10', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '10', '--max_iter', '400'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'COMPUTE-HYPER', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'en

In [10]:
print(best_hyperdrive_run.id)
print(best_hyperdrive_run.get_metrics()["Max iterations:"])


HD_83c46fb8-a462-4bff-818c-a1454623c5a4_7
400


In [11]:
#TODO: Save the best model
# Save and download the model
print(f'Run files: {best_hyperdrive_run.get_file_names()}')

Run files: ['azureml-logs/20_image_build_log.txt', 'azureml-logs/55_azureml-execution-tvmps_79ba0bafd193d90b2c08d6ebab99ea7b931778dec8a9371447009e851d47fdd1_d.txt', 'azureml-logs/65_job_prep-tvmps_79ba0bafd193d90b2c08d6ebab99ea7b931778dec8a9371447009e851d47fdd1_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_79ba0bafd193d90b2c08d6ebab99ea7b931778dec8a9371447009e851d47fdd1_d.txt', 'logs/azureml/94_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/best_hyper.pkl']


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
model = best_hyperdrive_run.register_model(model_name='breast-cancer-hyper', 
                                           model_path = os.path.join('outputs', 'best_hyper.pkl'),
                                           description='Breast Cancer detection using Azure HyperParameter',
                                           tags={'Method of execution':'HyperParameter'},
                                           properties={'Accuracy':best_hyperdrive_run.get_metrics()["Accuracy"]})

In [16]:
environment.save_to_directory(path='env', overwrite=False)

In [17]:
%%writefile hyper_scoring.py

import json
import numpy as np
import joblib
import pandas as pd
import os

from azureml.core.model import Model
def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'best_hyper.pkl')
    model = joblib.load(model_path)

def run(data): 
    try:
        data = np.array(json.loads(data))
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error


Writing hyper_scoring.py


In [27]:
inference_config = InferenceConfig(
    environment=environment,
    source_directory='.',
    entry_script='hyper_scoring.py',
)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [28]:
service = Model.deploy(workspace=ws,
                       name='breast-cancer-hyper-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-04 00:05:29+00:00 Creating Container Registry if not exists.
2021-11-04 00:05:29+00:00 Registering the environment.
2021-11-04 00:05:31+00:00 Use the existing image.
2021-11-04 00:05:31+00:00 Generating deployment configuration.
2021-11-04 00:05:33+00:00 Submitting deployment to compute.
2021-11-04 00:05:40+00:00 Checking the status of deployment breast-cancer-hyper-service..
2021-11-04 00:08:38+00:00 Checking the status of inference endpoint breast-cancer-hyper-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [29]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


In [30]:
scoring_uri = service.scoring_uri

TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:

score_data = [
    [9.504, 12.44, 60.34, 273.9, 0.1024],
    [15.37, 22.76, 100.2, 728.2, 0.092], 
    [21.09, 26.57, 142.7, 1311.0, 0.1141],
    [11.04, 14.93, 70.67, 372.7, 0.07987]]

In [32]:
import requests
import json

headers = {'Content-Type': 'application/json'}
score_data = json.dumps(score_data)
resp = requests.post(scoring_uri, score_data, headers=headers)
print(f'{resp.status_code}')
print(f'{resp.text}')



200
[1, 0, 0, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [33]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)


2021-11-04T00:08:19,051158000+00:00 - rsyslog/run 
2021-11-04T00:08:19,049752400+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-11-04T00:08:19,062104500+00:00 - nginx/run 
2021-11-04T00:08:19,065986800+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-11-04T00:08:19,473921500+00:00 - iot-server/finish 1 0
2021-11-04T00:08:19,480118100+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (73)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 98
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-11-04 00:08:23,012 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-11-04 00:08:23,014 | root | INFO | Starting up request id generator
2021-11-04 00:08:23,014 | root | INFO | Star

In [25]:
# Clean up any resources
# Delete the Webservice
# delete the compute cluster

service.delete()
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"



**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

